In [1]:
import pandas as pd
import numpy as np
import keras as keras
nombre_prueba = "0013-HSRn-CMCMCMFDrDD-0013"

path_npy = "DatasetsIAO/Unido/"
unido_train_X = np.load(path_npy + "unido_train_X.npy")
unido_train_Y = np.load(path_npy + "unido_train_Y.npy")
unido_test_X = np.load(path_npy + "unido_test_X.npy")
unido_test_Y = np.load(path_npy + "unido_test_Y.npy")
unido_val_X = np.load(path_npy + "unido_val_X.npy")
unido_val_Y = np.load(path_npy + "unido_val_Y.npy")

Using TensorFlow backend.
/home/david/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/david/.local/lib/python3.6/site-packages/tensorflow/python/fra

In [2]:
print(unido_train_X.shape)
print(unido_train_Y.shape)

print(unido_test_X.shape)
print(unido_test_Y.shape)

print(unido_val_X.shape)
print(unido_val_Y.shape)

(29437, 56, 56, 1)
(29437, 36)
(9812, 56, 56, 1)
(9812, 36)
(9812, 56, 56, 1)
(9812, 36)


In [3]:
from keras import models
from keras import layers

# Modelo de red neuronal
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(56, 56, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(36, activation='softmax'))
          
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 56, 56, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 28, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 14, 14, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
__________

In [4]:
from keras.preprocessing.image import ImageDataGenerator

# No normalizamos porque ya las hemos normalizado en el preprocesado.
train_datagen = ImageDataGenerator(
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     shear_range=0.2,
#     zoom_range=0.1
)

val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow(unido_train_X, unido_train_Y, batch_size=32)
val_generator = val_datagen.flow(unido_val_X, unido_val_Y, batch_size=32)

from keras import optimizers

model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adadelta(), 
              metrics=['acc'])


In [5]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=unido_train_X.shape[0]//32,
    epochs=1,
    validation_data=val_generator,
    validation_steps=unido_val_X.shape[0]//32
    )


Epoch 1/1
273/919 [=======>......................] - ETA: 1:24 - loss: 2.0502 - acc: 0.3886

KeyboardInterrupt: 

In [ ]:
model.save(nombre_prueba + '-mod.h5')

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.clf()
plt.grid(linestyle='-', linewidth=1.6, alpha=0.3)
plt.plot(epochs, acc, 'b', linewidth=1.0, label='Training acc')
plt.plot(epochs, val_acc, 'r', linewidth=1.0, label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
# plt.show()
plt.savefig(nombre_prueba + "-acc.png")

plt.clf()
plt.grid(linestyle='-', linewidth=1.6, alpha=0.3)
plt.plot(epochs, loss, 'b', linewidth=1.0, label='Training loss')
plt.plot(epochs, val_loss, 'r', linewidth=1.0, label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
# plt.show()
plt.savefig(nombre_prueba + "-loss.png")

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import cv2
import keras
import numpy as np
from sklearn.metrics import  confusion_matrix, accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# model = keras.models.load_model("modelo.h5")

model.save(nombre_prueba + '-mod.h5')

# Paso el one hot encoding a integer
labels = np.where(unido_test_Y==1)[1]

unido_test_X = unido_test_X.astype('float32')
unido_test_X /= 255.0

# Hago el predict en el conjunto de test
pred = model.predict(unido_test_X, verbose=1)

#pred = np.round(pred)
pred = np.argmax(pred, axis=1)

#Ahora puedo hacer la matriz de confusion
cm = confusion_matrix(labels, pred)

cm_sn = pd.DataFrame(cm)

plt.figure(figsize=(20,10))
sns.heatmap(cm_sn, annot=True, cmap='Blues', fmt='g')
plt.title('Accuracy:{0:.3f}'.format(accuracy_score(labels, pred)))
plt.ylabel('True value')
plt.xlabel('Predicted value')
# plt.show()
plt.savefig(nombre_prueba + "-MC.png")
print("Test acc: " + str(accuracy_score(labels, pred)))